In [1]:
#1) 导入常用的包
import nltk
from nltk.stem.lancaster import LancasterStemmer
import os
import json
import datetime
stemmer = LancasterStemmer()

In [3]:
#2) 构造3类意图：greeting, goodbye, sandwich:
training_data = []
training_data.append({"class":"greeting", "sentence":"how are you?" })
training_data.append({"class":"greeting", "sentence":"how is your day?" })
training_data.append({"class":"greeting", "sentence":"good day" })
training_data.append({"class":"greeting", "sentence":"how is it going today?" })

training_data.append({"class":"goodbye", "sentence":"have a nice day" })
training_data.append({"class":"goodbye", "sentence":"see you later" })
training_data.append({"class":"goodbye", "sentence":"have a nice day" })
training_data.append({"class":"goodbye", "sentence":"talk to you soon" })

training_data.append({"class":"sandwich", "sentence":"make me a sandwich" })
training_data.append({"class":"sandwich", "sentence":"can you make a sandwich?" })
training_data.append({"class":"sandwich", "sentence":"having a sandwich today?" })
training_data.append({"class":"sandwich", "sentence":"what's for lunch?" })
print("% sentences in training data" % len(training_data))


12entences in training data


In [4]:
training_data

[{'class': 'greeting', 'sentence': 'how are you?'},
 {'class': 'greeting', 'sentence': 'how is your day?'},
 {'class': 'greeting', 'sentence': 'good day'},
 {'class': 'greeting', 'sentence': 'how is it going today?'},
 {'class': 'goodbye', 'sentence': 'have a nice day'},
 {'class': 'goodbye', 'sentence': 'see you later'},
 {'class': 'goodbye', 'sentence': 'have a nice day'},
 {'class': 'goodbye', 'sentence': 'talk to you soon'},
 {'class': 'sandwich', 'sentence': 'make me a sandwich'},
 {'class': 'sandwich', 'sentence': 'can you make a sandwich?'},
 {'class': 'sandwich', 'sentence': 'having a sandwich today?'},
 {'class': 'sandwich', 'sentence': "what's for lunch?"}]

In [12]:
a = []
for i in training_data:
    print(i['class'])
    w = nltk.word_tokenize(i['sentence'])
    print(w)
    print((w, i['class']))
#     print(i)
#     print(i['sentence'])
#     a.append(i['sentence'])


greeting
['how', 'are', 'you', '?']
(['how', 'are', 'you', '?'], 'greeting')
greeting
['how', 'is', 'your', 'day', '?']
(['how', 'is', 'your', 'day', '?'], 'greeting')
greeting
['good', 'day']
(['good', 'day'], 'greeting')
greeting
['how', 'is', 'it', 'going', 'today', '?']
(['how', 'is', 'it', 'going', 'today', '?'], 'greeting')
goodbye
['have', 'a', 'nice', 'day']
(['have', 'a', 'nice', 'day'], 'goodbye')
goodbye
['see', 'you', 'later']
(['see', 'you', 'later'], 'goodbye')
goodbye
['have', 'a', 'nice', 'day']
(['have', 'a', 'nice', 'day'], 'goodbye')
goodbye
['talk', 'to', 'you', 'soon']
(['talk', 'to', 'you', 'soon'], 'goodbye')
sandwich
['make', 'me', 'a', 'sandwich']
(['make', 'me', 'a', 'sandwich'], 'sandwich')
sandwich
['can', 'you', 'make', 'a', 'sandwich', '?']
(['can', 'you', 'make', 'a', 'sandwich', '?'], 'sandwich')
sandwich
['having', 'a', 'sandwich', 'today', '?']
(['having', 'a', 'sandwich', 'today', '?'], 'sandwich')
sandwich
['what', "'s", 'for', 'lunch', '?']
(['what'

In [21]:
#3) 进行数据预处理
words = []
classes = []
documents = []
ignore_words = ['?']

#遍历训练集合
for pattern in training_data:
    #使用nltk.tokenize进行分词
    w = nltk.word_tokenize(pattern['sentence'])
    #把分词后的sentence加入words中
    words.extend(w)
    #把class加入documents中
    documents.append((w, pattern['class']))
    #把不在classes list中的词语加入classes
    if pattern['class'] not in classes:
        classes.append(pattern['class'])
        
    #取词干，小写，去重
    words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
    words = list(set(words))
    classes = list(set(classes))
    
    print(len(documents), "documents长度",documents)
    print(len(classes), "classes的长度", classes)
    print(len(words), "words的长度", words)
    print("-------------------------")
    

1 documents长度 [(['how', 'are', 'you', '?'], 'greeting')]
1 classes的长度 ['greeting']
3 words的长度 ['how', 'you', 'ar']
-------------------------
2 documents长度 [(['how', 'are', 'you', '?'], 'greeting'), (['how', 'is', 'your', 'day', '?'], 'greeting')]
1 classes的长度 ['greeting']
6 words的长度 ['ar', 'you', 'day', 'is', 'how', 'yo']
-------------------------
3 documents长度 [(['how', 'are', 'you', '?'], 'greeting'), (['how', 'is', 'your', 'day', '?'], 'greeting'), (['good', 'day'], 'greeting')]
1 classes的长度 ['greeting']
7 words的长度 ['ar', 'you', 'good', 'day', 'is', 'how', 'yo']
-------------------------
4 documents长度 [(['how', 'are', 'you', '?'], 'greeting'), (['how', 'is', 'your', 'day', '?'], 'greeting'), (['good', 'day'], 'greeting'), (['how', 'is', 'it', 'going', 'today', '?'], 'greeting')]
1 classes的长度 ['greeting']
10 words的长度 ['it', 'ar', 'you', 'good', 'going', 'day', 'is', 'how', 'yo', 'today']
-------------------------
5 documents长度 [(['how', 'are', 'you', '?'], 'greeting'), (['how', 'is',

In [31]:
#4） 将训练数据中的每个句子转化为词袋，每个训练语句变化为只有0和1的数组
training = []
output = []

output_empty = [0] * len(classes)
for doc in documents:
    #初始化词袋
    bag = []
    #取doc中第一个元素，就是分词后结果
    pattern_words = doc[0]
    #stem+小写
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    #创建words array, words（已经set过的），让每句实现0、1表示
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
#         print(bag)
    
#     print("--------------------")
    training.append(bag)
#     print(training)
#     print("********************")
    
    #表示输出哪一个意图的类
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    output.append(output_row)

i = 0
w = documents[i][0]
print(w)
print([stemmer.stem(word.lower()) for word in w])
print("training中第一个元素：",training[i])
print("classes类别:",output[i])
print("----------------------")
print("# words", len(words))
print("# classes", len(classes))
    


['how', 'are', 'you', '?']
['how', 'ar', 'you', '?']
training中第一个元素： [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
classes类别: [1, 0, 0]
----------------------
# words 26
# classes 3


In [32]:
#5） 使用Sigmoid函数作为神经元的激活函数。然后不断进行参数调整，直到错误率降低到可接受
import numpy as np
import time

#定义sigmoid函数
def sigmoid(x):
    output = 1/(1+np.exp(-x))
    return output

# convert output of sigmoid function to its derivative
def sigmoid_output_to_derivative(output):
    return output*(1-output)

def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

#返回bag of words, 0,1对每个sentence中的word
def bow(sentence, words, show_details = False):
    sentence_words = clean_up_sentence(sentence)
    bag = [0] * len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print("found in bag: %s" % w )
    return (np.array(bag))

def think(sentence, show_details = False):
    x = bow(sentence.lower(), words, show_details)
    if show_details:
        print("sentence:", sentence, "\n bow:", x)
    # input layer is our bag of words
    l0 = x
    # matrix multiplication of input and hidden layer
    l1 = sigmoid(np.dot(l0, synapse_0))
    # output layer
    l2 = sigmoid(np.dot(l1, synapse_1))
    return l2


In [35]:
def train(X, y, hidden_neurons=10, alpha=1, epochs=50000, dropout=False, dropout_percent=0.5):

    print ("Training with %s neurons, alpha:%s, dropout:%s %s" % (hidden_neurons, str(alpha), dropout, dropout_percent if dropout else '') )
    print ("Input matrix: %sx%s    Output matrix: %sx%s" % (len(X),len(X[0]),1, len(classes)) )
    np.random.seed(1)

    last_mean_error = 1
    # randomly initialize our weights with mean 0
    synapse_0 = 2*np.random.random((len(X[0]), hidden_neurons)) - 1
    synapse_1 = 2*np.random.random((hidden_neurons, len(classes))) - 1

    prev_synapse_0_weight_update = np.zeros_like(synapse_0)
    prev_synapse_1_weight_update = np.zeros_like(synapse_1)

    synapse_0_direction_count = np.zeros_like(synapse_0)
    synapse_1_direction_count = np.zeros_like(synapse_1)
        
    for j in iter(range(epochs+1)):

        # Feed forward through layers 0, 1, and 2
        layer_0 = X
        layer_1 = sigmoid(np.dot(layer_0, synapse_0))
                
        if(dropout):
            layer_1 *= np.random.binomial([np.ones((len(X),hidden_neurons))],1-dropout_percent)[0] * (1.0/(1-dropout_percent))

        layer_2 = sigmoid(np.dot(layer_1, synapse_1))

        # how much did we miss the target value?
        layer_2_error = y - layer_2

        if (j% 10000) == 0 and j > 5000:
            # if this 10k iteration's error is greater than the last iteration, break out
            if np.mean(np.abs(layer_2_error)) < last_mean_error:
                print ("delta after "+str(j)+" iterations:" + str(np.mean(np.abs(layer_2_error))) )
                last_mean_error = np.mean(np.abs(layer_2_error))
            else:
                print ("break:", np.mean(np.abs(layer_2_error)), ">", last_mean_error )
                break
                
        # in what direction is the target value?
        # were we really sure? if so, don't change too much.
        layer_2_delta = layer_2_error * sigmoid_output_to_derivative(layer_2)

        # how much did each l1 value contribute to the l2 error (according to the weights)?
        layer_1_error = layer_2_delta.dot(synapse_1.T)

        # in what direction is the target l1?
        # were we really sure? if so, don't change too much.
        layer_1_delta = layer_1_error * sigmoid_output_to_derivative(layer_1)
        
        synapse_1_weight_update = (layer_1.T.dot(layer_2_delta))
        synapse_0_weight_update = (layer_0.T.dot(layer_1_delta))
        
        if(j > 0):
            synapse_0_direction_count += np.abs(((synapse_0_weight_update > 0)+0) - ((prev_synapse_0_weight_update > 0) + 0))
            synapse_1_direction_count += np.abs(((synapse_1_weight_update > 0)+0) - ((prev_synapse_1_weight_update > 0) + 0))        
        
        synapse_1 += alpha * synapse_1_weight_update
        synapse_0 += alpha * synapse_0_weight_update
        
        prev_synapse_0_weight_update = synapse_0_weight_update
        prev_synapse_1_weight_update = synapse_1_weight_update

    now = datetime.datetime.now()

    # persist synapses
    synapse = {'synapse0': synapse_0.tolist(), 'synapse1': synapse_1.tolist(),
               'datetime': now.strftime("%Y-%m-%d %H:%M"),
               'words': words,
               'classes': classes
              }
    synapse_file = "synapses.json"

    with open(synapse_file, 'w') as outfile:
        json.dump(synapse, outfile, indent=4, sort_keys=True)
    print ("saved synapses to:", synapse_file)
        

In [36]:
X = np.array(training)
y = np.array(output)

start_time = time.time()

train(X, y, hidden_neurons=20,alpha=0.1, epochs = 100000, dropout=False, dropout_percent=0.2)

elapsed_time = time.time() - strat_time
print("Processing time", elapsed_time, "seconds")

Training with 20 neurons, alpha:0.1, dropout:False 
Input matrix: 12x26    Output matrix: 1x3
delta after 10000 iterations:0.006339386725862213
delta after 20000 iterations:0.004323590964018436
delta after 30000 iterations:0.0034661910912994302
delta after 40000 iterations:0.00296613750702867
delta after 50000 iterations:0.0026298645114846285
delta after 60000 iterations:0.0023843391538275723
delta after 70000 iterations:0.00219515895992727
delta after 80000 iterations:0.002043744113821668
delta after 90000 iterations:0.0019190751729039777
delta after 100000 iterations:0.0018141535422373021
saved synapses to: synapses.json


NameError: name 'strat_time' is not defined

In [37]:
# probability threshold
ERROR_THRESHOLD = 0.2
# load our calculated synapse values
synapse_file = 'synapses.json' 
with open(synapse_file) as data_file: 
    synapse = json.load(data_file) 
    synapse_0 = np.asarray(synapse['synapse0']) 
    synapse_1 = np.asarray(synapse['synapse1'])

def classify(sentence, show_details=False):
    results = think(sentence, show_details)

    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD ] 
    results.sort(key=lambda x: x[1], reverse=True) 
    return_results =[[classes[r[0]],r[1]] for r in results]
    print ("%s \n classification: %s" % (sentence, return_results))
    return return_results

In [38]:
classify("how are you today?")

how are you today? 
 classification: [['greeting', 0.9985682149776903]]


[['greeting', 0.9985682149776903]]

In [39]:
classify("how about your lunch")

how about your lunch 
 classification: [['greeting', 0.9032603255315935], ['sandwich', 0.3469762186731649]]


[['greeting', 0.9032603255315935], ['sandwich', 0.3469762186731649]]

In [41]:
classify("make me some lunch")

make me some lunch 
 classification: [['sandwich', 0.9906203898490786]]


[['sandwich', 0.9906203898490786]]

In [42]:
classify("talk to you tomorrow?")

talk to you tomorrow? 
 classification: [['goodbye', 0.99030023609869]]


[['goodbye', 0.99030023609869]]